In [4]:
import pandas as pd
mismatched_df=pd.read_csv('../../results/predictions/CMNPD2.0_test_set_predict_mismatched.csv')
mismatched_df

,cid,smiles,oid,origin,label,pred
0,CMNPD31563,C12N(CCC1)C(=O)C(C(C)CC)NC(=O)C3N(CCC3)C(=O)C(...,CMNPD372185,Bacteria,1,2
1,CMNPD31566,C1(=O)[C@H](CC(=O)N)NC(=O)[C@H](C)NC(=O)[C@H](...,CMNPD372133,Bacteria,1,0
2,CMNPD31567,N(C(=O)CC(O)C(C)N)[C@@H]([C@H](O)C(=O)O)C(=O)N...,CMNPD373615,Bacteria,1,0
3,CMNPD31577,C1(C)=CN(C2C[C@H](O)[C@@H](CO)O2)C(=O)N(SNC(=O...,CMNPD372185,Bacteria,1,0
4,CMNPD31578,C1(C)=CN(C2C[C@H](O)[C@@H](CO)O2)C(=O)N(SN)C1=O,CMNPD372185,Bacteria,1,0
...,...,...,...,...,...,...
715,CMNPD38347,c1cc(c(CC(=O)O[C@H](C)[C@H](C)O)c[nH]2)c2cc1,CMNPD372956,Fungi,2,1
716,CMNPD38348,c1cc(c(CC(=O)O[C@@H](C)[C@H](C)O)c[nH]2)c2cc1,CMNPD372956,Fungi,2,1
717,CMNPD38349,C\C=C\C=C\C(=O)OC(C(C)C(CC)=O)CC(O)C(OC)=O,CMNPD373727,Fungi,2,0
718,CMNPD38354,O[C@H](C(C)C)C(=O)N[C@@H]([C@H](C)CC)C(=O)O[C@...,CMNPD372965,Fungi,2,1


In [5]:
mismatched_df['origin'].value_counts()

origin
Fungi       279
Bacteria    244
Animalia    197
Name: count, dtype: int64

In [6]:
Bacteria_to_Animalia = mismatched_df[(mismatched_df["label"] == 1) & (mismatched_df["pred"] == 0)]

Bacteria_to_Animalia

,cid,smiles,oid,origin,label,pred
1,CMNPD31566,C1(=O)[C@H](CC(=O)N)NC(=O)[C@H](C)NC(=O)[C@H](...,CMNPD372133,Bacteria,1,0
2,CMNPD31567,N(C(=O)CC(O)C(C)N)[C@@H]([C@H](O)C(=O)O)C(=O)N...,CMNPD373615,Bacteria,1,0
3,CMNPD31577,C1(C)=CN(C2C[C@H](O)[C@@H](CO)O2)C(=O)N(SNC(=O...,CMNPD372185,Bacteria,1,0
4,CMNPD31578,C1(C)=CN(C2C[C@H](O)[C@@H](CO)O2)C(=O)N(SN)C1=O,CMNPD372185,Bacteria,1,0
33,CMNPD31702,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,CMNPD372299,Bacteria,1,0
...,...,...,...,...,...,...
612,CMNPD37727,[C@@]1([H])(C(C)CC)C(=O)N[C@@H](Cc2cc(Cl)c(O)c...,CMNPD372293,Bacteria,1,0
613,CMNPD37728,c1(C[C@@H](C(=O)O)N=C2C)c2[nH]c(cc[nH]3)c13,CMNPD372301,Bacteria,1,0
614,CMNPD37735,C1(C(CCCCC)O1)C(O)CCCCCCCCCC(=O)O,CMNPD373608,Bacteria,1,0
621,CMNPD37759,[C@@H]([C@H](CC)C)(N)C(=O)N[C@@H](C)C(=O)N[C@@...,CMNPD373619,Bacteria,1,0


In [7]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdFingerprintGenerator, DataStructs

train_data_path = "../../data/processed/data_cmnpd_after2000.csv"

train_df = pd.read_csv(train_data_path)

morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2, fpSize=2048)

def get_fingerprint(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        return morgan_gen.GetFingerprint(mol) 
    return None

Bacteria_to_Animalia["Fingerprint"] = Bacteria_to_Animalia["smiles"].apply(get_fingerprint)
train_df["Fingerprint"] = train_df["SMILES"].apply(get_fingerprint)

bacteria_to_animalia = Bacteria_to_Animalia.dropna(subset=["Fingerprint"])
train_df = train_df.dropna(subset=["Fingerprint"])

matches = []

for idx, row in bacteria_to_animalia.iterrows():
    query_fp = row["Fingerprint"]
    query_smiles = row["smiles"]
    
    for _, train_row in train_df.iterrows():
        train_fp = train_row["Fingerprint"]
        similarity = DataStructs.TanimotoSimilarity(query_fp, train_fp)
        
        if similarity >= 0.7:
            matches.append({
                "Query_SMILES": query_smiles,
                "Matched_Train_SMILES": train_row["SMILES"],
                "Similarity": similarity,
                "Train_Label": train_row["labels"] 
            })

matches_df = pd.DataFrame(matches)


matches_df

/tmp/ipykernel_702/3358753199.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bacteria_to_Animalia["Fingerprint"] = Bacteria_to_Animalia["smiles"].apply(get_fingerprint)


,Query_SMILES,Matched_Train_SMILES,Similarity,Train_Label
0,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c1ccc2c(n3c(c(n([C@]([H])(O[C@@]3(C)[C@@H]4OC)...,0.701299,0
1,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c1ccc2c(n3c(c(n([C@]([H])(O[C@@]3(C)[C@@H]4OC)...,0.710526,0
2,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c1ccc2c(n3c(c(n([C@]([H])(O[C@@]3(C)[C@@H]4O)C...,0.743243,0
3,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c1cc2c(cc1O)n([C@@H]3O[C@@]4(C)[C@H](OC)[C@H](...,0.722222,0
4,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c1(cc2c(cc1)n([C@@H]3O[C@@]4(C)[C@H](OC)[C@H](...,0.726027,0
5,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c12c(cccc1)c3c(c(c(c4c3C(=O)NC4)c5c6cccc5)n6[C...,0.702703,1
6,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,N(C)([C@H]1C[C@]2([H])O[C@@](C)(n3c(c(c(c(C(NC...,0.710526,1
7,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c12c(cccc1)n3c(c(c(c4c5C(=O)NC4)c6c7cccc6)n7[C...,0.701299,1
8,C1c(c2c(n([C@](C)([C@H](OC)[C@@H]3NC)O[C@@H]4C...,c12c(cccc1)n3c(c(c(c4c5C(=O)NC4)c6c7cccc6)n7[C...,0.720000,1
9,c1(nc(OC)c(OC)c(O)c1C)C\C=C(/C)\C\C(\C)=C\C(\C...,c1(c(nc(C\C=C(\C\C(=C\C(=C\C(C(O)\C(=C\C)\C)C)...,0.844828,1
